In [1]:
import sys
print(sys.version)
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

2.7.12rc1 (v2.7.12rc1:13912cd1e7e8, Jun 11 2016, 15:32:34) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]


In [2]:
from pyspark.mllib.regression import LabeledPoint
def transform_data(data):
    prediction_col = "failure"
    columns = []#["serial_number", "model", "capacity_bytes"]
    # Add Smart columns.
    for i in range(1, 255):
        test_col = "smart_" + str(i) + "_normalized"
        try:
            data[test_col]
            columns.append(test_col)
        except:
            pass
    return (float(data[prediction_col]), [0.0 if data[i] is None else data[i] for i in columns])

In [3]:
from pyspark.conf import *
spark = SparkSession.builder.master("local[*]").appName("SENG474-HD").getOrCreate()
df = spark.read.load('data_Q1_2017/2017-03-2*.csv', 
                     format='com.databricks.spark.csv', 
                     header='true', 
                     inferSchema='true')

#data = df.filter(col("failure") == 1).rdd.map(transform_data).collect()
data = df.rdd.map(transform_data).collect()

labels, data = zip(*data)
print(data[0], type(data), labels[0], type(labels))

([100, 135, 127, 100, 100, 100, 122, 95, 100, 0.0, 100, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100, 100, 253, 0.0, 100, 100, 100, 200, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], <type 'tuple'>, 0.0, <type 'tuple'>)


In [4]:
print(len(labels))

838240


In [5]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.3)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(data_train, labels_train)
predictions = clf.predict(data_test)
labels_and_predictions = zip(predictions, labels_test)
testErr = float(len(filter(lambda lp: lp[0] != lp[1], labels_and_predictions))) / float(len(data_test))
print("SKLean Test Error: {0:03f}%".format(testErr * 100.0))

SKLean Test Error: 0.009941%


In [10]:
print(labels_and_predictions[0])

(0.0, 0.0)


In [8]:
df.filter(col("failure") == 1).show(10)

+-------------------+--------------+--------------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------------